In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

2024-05-10 13:27:19.198327: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.13.1


In [2]:
# Wczytaj dane treningowe i testowe
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

train_set = pd.read_csv('adult/adult.data', sep=", ",header = None)
test_set = pd.read_csv('adult/adult.test', sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

train = train_set.replace('?', np.nan).dropna()
test = test_set.replace('?', np.nan).dropna()

train_set.head()

/var/folders/k_/2tytwy6d2ll1_kvk_k0j7vd00000gn/T/ipykernel_8394/4204620386.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_set = pd.read_csv('adult/adult.data', sep=", ",header = None)
/var/folders/k_/2tytwy6d2ll1_kvk_k0j7vd00000gn/T/ipykernel_8394/4204620386.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_set = pd.read_csv('adult/adult.test', sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
dataset = pd.concat([train,test])

dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0,'<=50K':0, '>50K.':1, '>50K':1})

dataset.drop(["fnlwgt"],axis=1,inplace=True)

dataset.drop(["education"],axis=1,inplace=True)

x = dataset.groupby('native_country')["wage_class"].mean()

d = dict(pd.cut(x[x.index!=" United-States"],5,labels=range(5)))

dataset['native_country'] = dataset['native_country'].replace(d)

dataset = pd.get_dummies(dataset,drop_first=True)

train = dataset.iloc[:train.shape[0]]
test = dataset.iloc[train.shape[0]:]

X_train = train.drop("wage_class",axis=1)
y_train = train.wage_class

X_test = test.drop("wage_class",axis=1)
y_test = test.wage_class

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

print(X_train.shape)
print(X_test.shape)

(30162, 41)
(15060, 41)


In [4]:
keras.backend.clear_session()

def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[41,]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [13]:
!pip install scikeras 
from scikeras.wrappers import KerasClassifier 
keras_class = KerasClassifier(build_model)
keras_class


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


KerasClassifier(
	model=<function build_model at 0x7fbc88bf4790>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	n_neurons=41
	n_hidden=2
	class_weight=None
)

In [14]:
from keras.callbacks import EarlyStopping
keras_class.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)])

Epoch 1/10
849/849 [==============================] - 2s 1ms/step - loss: 1.3081 - accuracy: 0.7871 - val_loss: 3.9316 - val_accuracy: 0.7451
Epoch 2/10
849/849 [==============================] - 1s 1ms/step - loss: 3.8290 - accuracy: 0.7517 - val_loss: 3.9316 - val_accuracy: 0.7451
Epoch 3/10
849/849 [==============================] - 1s 1ms/step - loss: 3.8289 - accuracy: 0.7517 - val_loss: 3.9316 - val_accuracy: 0.7451
Epoch 4/10
849/849 [==============================] - 1s 1ms/step - loss: 3.8289 - accuracy: 0.7517 - val_loss: 3.9316 - val_accuracy: 0.7451
Epoch 5/10
849/849 [==============================] - 1s 1ms/step - loss: 3.8289 - accuracy: 0.7517 - val_loss: 3.9316 - val_accuracy: 0.7451
Epoch 6/10
849/849 [==============================] - 1s 1ms/step - loss: 3.8289 - accuracy: 0.7517 - val_loss: 3.9316 - val_accuracy: 0.7451
Epoch 7/10
849/849 [==============================] - 1s 1ms/step - loss: 3.8289 - accuracy: 0.7517 - val_loss: 3.9316 - val_accuracy: 0.7451
Epoch 

KerasClassifier(
	model=<function build_model at 0x7fbc88bf4790>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	n_neurons=41
	n_hidden=2
	class_weight=None
)

# Zad

Wykonajmy RandomizedSearchCV

In [15]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100)
}


Fitting 3 folds for each of 10 candidates, totalling 30 fits


Epoch 1/100
566/566 [==============================] - 1s 2ms/step - loss: 11.1039 - accuracy: 0.2638 - val_loss: 11.2227 - val_accuracy: 0.2640
Epoch 2/100
566/566 [==============================] - 1s 1ms/step - loss: 11.4785 - accuracy: 0.2473 - val_loss: 11.2227 - val_accuracy: 0.2640
Epoch 3/100
566/566 [==============================] - 1s 1ms/step - loss: 11.4785 - accuracy: 0.2473 - val_loss: 11.2227 - val_accuracy: 0.2640
Epoch 4/100
566/566 [==============================] - 1s 1ms/step - loss: 11.4785 - accuracy: 0.2473 - val_loss: 11.2227 - val_accuracy: 0.2640
Epoch 5/100
566/566 [==============================] - 1s 1ms/step - loss: 11.4785 - accuracy: 0.2473 - val_loss: 11.2227 - val_accuracy: 0.2640
Epoch 6/100
566/566 [==============================] - 1s 1ms/step - loss: 11.4785 - accuracy: 0.2473 - val_loss: 11.2227 - val_accuracy: 0.2640
Epoch 7/100
566/566 [==============================] - 1s 1ms/step - loss: 11.4785 - accuracy: 0.2473 - val_loss: 11.2227 - val_ac

Epoch 1/100
566/566 [==============================] - 1s 2ms/step - loss: 1.9946 - accuracy: 0.7801 - val_loss: 4.0729 - val_accuracy: 0.7360
Epoch 2/100
566/566 [==============================] - 1s 1ms/step - loss: 3.8134 - accuracy: 0.7528 - val_loss: 4.0729 - val_accuracy: 0.7360
Epoch 3/100
566/566 [==============================] - 1s 1ms/step - loss: 3.8134 - accuracy: 0.7528 - val_loss: 4.0729 - val_accuracy: 0.7360
Epoch 4/100
566/566 [==============================] - 1s 1ms/step - loss: 3.8134 - accuracy: 0.7528 - val_loss: 4.0729 - val_accuracy: 0.7360
Epoch 5/100
566/566 [==============================] - 1s 1ms/step - loss: 3.8134 - accuracy: 0.7528 - val_loss: 4.0729 - val_accuracy: 0.7360
Epoch 6/100
566/566 [==============================] - 1s 1ms/step - loss: 3.8134 - accuracy: 0.7528 - val_loss: 4.0729 - val_accuracy: 0.7360
Epoch 7/100
566/566 [==============================] - 1s 1ms/step - loss: 3.8134 - accuracy: 0.7528 - val_loss: 4.0729 - val_accuracy: 0.7360


KeyboardInterrupt



In [ ]:
rnd_search_cv.best_params_

# Zad

Wykonaj RandomizedSearchCV na danych FashionMINT
